# Run a CESM case

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from subprocess import check_call
import cesm_tools

from project import *

In [3]:
clobber = True

refcase = "g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005"
refdate = "0347-01-01"
refcaserest_root = f"{dir_project_root}/data/{refcase}/rest"

compset = "OMIP_DATM%JRA-1p4-2018_SLND_CICE_POP2%ECO_DROF%JRA-1p4-2018_SGLC_WW3_SIAC_SESP"
res = "TL319_g17"

case = f"smyle.control.{mach}.003"

stop_n = 15
stop_option = "nyear"
resubmit = 0

queue = "regular"

wallclock = "00:30:00" if queue == "debug" else "24:00:00"    

In [4]:
caseroot = f"{dir_caseroot_root}/{case}"
assert not os.path.exists(caseroot) or clobber, f"Case {case} exists; caseroot:\n{caseroot}\n"

rundir = f"{dir_scratch}/{case}"
archive_root = f"{dir_scratch}/archive/{case}"

check_call(["rm", "-fr", caseroot])
check_call(["rm", "-fr", rundir])
check_call(["rm", "-fr", archive_root])

check_call(
    " ".join([
        "module load python",
        "&&",      
        "./create_newcase",
        "--compset", compset,
        "--case", caseroot,
        "--res", res,
        "--machine", mach,
        "--project", account,            
        "--run-unsupported"]),
    shell=True,
    cwd=f"{coderoot}/cime/scripts",
)

Did not find an alias or longname compset match for OMIP_DATM%JRA-1p4-2018_SLND_CICE_POP2%ECO_DROF%JRA-1p4-2018_SGLC_WW3_SIAC_SESP 
Compset forcing is 
ATM component is  Data driven ATM interannual JRA55 forcing, v1.4, through 2018
LND component is Stub land component
ICE component is Sea ICE (cice) model version 5
OCN component is POP2 Ecosystem
ROF component is Data runoff modelJRA55 interannual forcing, v1.4, through 2018
GLC component is Stub glacier (land ice) component
WAV component is Wave Watch
ESP component is 
Pes     specification file is /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/../components/pop//cime_config/config_pes.xml
Machine is perlmutter
Pes setting: compset_match is _DATM.*_POP2 
Pes setting: grid          is a%TL319_l%null_oi%gx1v7_r%JRA025v2_g%null_w%ww3a_m%gx1v7 
Pes setting: compset       is OMIP_DATM%JRA-1p4-2018_SLND_CICE_POP2%ECO_DROF%JRA-1p4-2018_SGLC_WW3_SIAC_SESP 
Pes setting: tasks       is {'NTASKS_OCN': -1, 'NTASKS_ICE': -1, 'NTASKS_CPL'

Could not find machine match for 'login29.chn' or 'login29'


0

In [5]:
def xmlchange(arg, force=False):
    """call xmlchange"""
    check_call(f"module load python && ./xmlchange {arg}", cwd=caseroot, shell=True)

In [6]:
xmlchange("MAX_TASKS_PER_NODE=128")
xmlchange("MAX_MPITASKS_PER_NODE=128")

xmlchange("NTASKS_ATM=72")
xmlchange("NTASKS_CPL=72")
xmlchange("NTASKS_WAV=72")
xmlchange("NTASKS_GLC=72")
xmlchange("NTASKS_ICE=72")
xmlchange("NTASKS_ROF=72")
xmlchange("NTASKS_LND=72")
xmlchange("NTASKS_ESP=72")

xmlchange("NTASKS_OCN=751")
xmlchange("ROOTPE_OCN=72")

xmlchange("CICE_BLCKX=16")
xmlchange("CICE_BLCKY=16")
xmlchange("CICE_MXBLCKS=7")
xmlchange("CICE_DECOMPTYPE='sectrobin'")
xmlchange("CICE_DECOMPSETTING='square-ice'")

xmlchange("OCN_TRACER_MODULES='iage ecosys'")

xmlchange("POP_AUTO_DECOMP=FALSE")
xmlchange("POP_BLCKX=9")
xmlchange("POP_BLCKY=16")
xmlchange("POP_NX_BLOCKS=36")
xmlchange("POP_NY_BLOCKS=24")
xmlchange("POP_MXBLCKS=1")
xmlchange("POP_DECOMPTYPE='spacecurve'")

In [7]:
# copy restarts
os.makedirs(f"{dir_scratch}/{case}/run", exist_ok=True)
check_call(
    f"cp -v {refcaserest_root}/{refdate}-00000/* {dir_scratch}/{case}/run/.",
    shell=True,
)

'/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/data/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/rest/0347-01-01-00000/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.cice.r.0347-01-01-00000.nc' -> '/pscratch/sd/m/mattlong/smyle.control.perlmutter.003/run/./g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.cice.r.0347-01-01-00000.nc'
'/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/data/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/rest/0347-01-01-00000/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.cpl.r.0347-01-01-00000.nc' -> '/pscratch/sd/m/mattlong/smyle.control.perlmutter.003/run/./g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.cpl.r.0347-01-01-00000.nc'
'/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/data/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/rest/0347-01-01-00000/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005.datm.rs1.0347-01-01-00000.bin' -> '/pscratch/sd/m/mattlong/smyle.control.perlmutter.003/run/./g.e2

0

In [8]:
xmlchange(f"DIN_LOC_ROOT={cesm_inputdata}")

check_call(
    "module load python && ./case.setup",
    cwd=caseroot,
    shell=True,
)

# handle the myserious failures with ESP
os.makedirs(f"{caseroot}/SourceMods/src.desp", exist_ok=True)
with open(f"{dir_scratch}/{case}/run/rpointer.esp", "w") as fid:
    fid.write("\n")

Setting resource.RLIMIT_STACK to -1 from (8388608, -1)
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None WALLTIME_FORMAT %H:%M:%S
Creating batch scripts
Writing case.run script from input template /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/config/cesm/machines/template.case.run


/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/env_mach_specific.xml already exists, delete to replace


Creating file .case.run
Writing case.st_archive script from input template /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
Creating user_nl_xxx files for components and cpl
If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run


In [9]:
check_call(
    f"cp -vr {scriptroot}/CESM-RefCase/{refcase}/SourceMods/* {caseroot}/SourceMods",
    shell=True,
)

'/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/SourceMods/src.cice/README' -> '/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/SourceMods/src.cice/README'
'/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/SourceMods/src.datm/README' -> '/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/SourceMods/src.datm/README'
'/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/SourceMods/src.drof/README' -> '/global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/SourceMods/src.drof/README'
'/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/SourceMods

0

In [10]:
user_nl_files = glob(f"{scriptroot}/CESM-RefCase/{refcase}/user_nl*")

for file in user_nl_files:
    file_out = os.path.join(caseroot, os.path.basename(file))
    print(f"{file} -> {file_out}")
    with open(file, "r") as fid:
        file_str = fid.read().replace("/glade/p/cesmdata/cseg/inputdata", cesm_inputdata)
    with open(file_out, "w") as fid:
        fid.write(file_str)

/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_nl_drof -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_nl_drof
/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_nl_pop -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_nl_pop
/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_nl_datm -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_nl_datm
/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_nl_ww -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.0

In [11]:
user_datm_files = glob(f"{scriptroot}/CESM-RefCase/{refcase}/user_datm.*")

for file in user_datm_files:
    file_out = os.path.join(caseroot, os.path.basename(file))
    print(f"{file} -> {file_out}")
    with open(file, "r") as fid:
        file_str = fid.read().replace("/glade/p/cesmdata/cseg/inputdata", cesm_inputdata)
    with open(file_out, "w") as fid:
        fid.write(file_str)

/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_datm.streams.txt.co2tseries.omip -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_datm.streams.txt.co2tseries.omip
/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_datm.streams.txt.presaero.cesm2_SMYLE -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_datm.streams.txt.presaero.cesm2_SMYLE
/global/u2/m/mattlong/codes/oae-dor-global-efficiency/notebooks/CESM-RefCase/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.SMYLE.005/user_datm.streams.txt.presaero.clim_1850 -> /global/cfs/projectdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003/user_datm.streams.txt.presaero.clim_1850


In [12]:
xmlchange(f"RUN_TYPE=branch")
xmlchange(f"RUN_STARTDATE={refdate}")
xmlchange(f"RUN_REFCASE={refcase}")
xmlchange(f"RUN_REFDATE={refdate}")

xmlchange(f"STOP_N={stop_n}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"REST_N={stop_n}")
xmlchange(f"REST_OPTION={stop_option}")
xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"JOB_WALLCLOCK_TIME={wallclock}")

xmlchange(f"CHARGE_ACCOUNT={account}")
xmlchange(f"PROJECT={account}")
xmlchange(f"JOB_QUEUE={queue}")

In [13]:
check_call(
    "module load python && ./case.build",
    cwd=caseroot,
    shell=True,
)

Building case in directory /global/cfs/cdirs/m4746/Projects/OAE-Efficiency-Map/cesm-cases/smyle.control.perlmutter.003
sharedlib_only is False
model_only is False
Setting resource.RLIMIT_STACK to -1 from (8388608, -1)
Generating component namelists as part of build
GET_REFCASE is false, the user is expected to stage the refcase to the run directory.
Creating component namelists
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/data_comps/datm/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/stub_comps/slnd/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/components/cice//cime_config/buildnml
     ...buildnml calling cice buildcpp to set build time options
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/components/pop//cime_config/buildnml
     ... buildnml: calling pop buildcpp to set build time options
   Calling /global/u2/m/mattlong/codes/release-

0

In [14]:
check_call(
    "module load python && ./case.submit",
    cwd=caseroot,
    shell=True,
)

Setting resource.RLIMIT_STACK to -1 from (8388608, -1)
GET_REFCASE is false, the user is expected to stage the refcase to the run directory.
Creating component namelists
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/data_comps/datm/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/stub_comps/slnd/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/components/cice//cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/components/pop//cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/data_comps/drof/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/cime/src/components/stub_comps/sglc/cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-cesm2.1.5-SMYLE/components/ww3//cime_config/buildnml
   Calling /global/u2/m/mattlong/codes/release-ce

submit_jobs case.run
Submit job case.run


Submitted job id is 30009267
Submitting job script sbatch --time 24:00:00 -q regular --account m4746  --dependency=afterok:30009267 case.st_archive --resubmit


Submit job case.st_archive


Submitted job id is 30009268
Submitted job case.run with id 30009267
Submitted job case.st_archive with id 30009268


0